In [ ]:
# === Step 1: Import Libraries ===
from google.colab import drive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from scipy import signal
import seaborn as sns
from sklearn.metrics import confusion_matrix
import zipfile
import shutil

# === Step 2: Extract Data ===
zip_file = "/content/train.zip"
extract_folder = "/content/trainimages"
csv_file = "/content/trainLabels.csv"

if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

try:
    with zipfile.ZipFile(zip_file, 'r') as archive:
        archive.extractall(extract_folder)
except zipfile.BadZipFile:
    shutil.unpack_archive(zip_file, extract_folder)

image_folder = "/content/trainimages/train"

# === Step 3: Load Dataset ===
df = pd.read_csv(csv_file)

def load_image(image_id, img_size=(28, 28)):
    img_path = os.path.join(image_folder, f"{image_id}.png")
    if os.path.exists(img_path):
        img = imread(img_path)
        img_resized = resize(img, img_size)
        img_gray = rgb2gray(img_resized)
        return img_gray
    return None

# === Step 4: Preprocess Data ===
X_data, y_data = [], []

for index, row in df.iterrows():
    img = load_image(row["id"])
    if img is not None:
        X_data.append(img)
        y_data.append(row["label"])

X_data = np.array(X_data).reshape(len(X_data), 28, 28, 1)
y_data = np.array(pd.factorize(y_data)[0])

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# === Step 5: Design CNN Model (A1) ===
model = Sequential([
    Conv2D(64, (3,3), input_shape=(28, 28, 1), activation='relu', name="conv1"),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu', name="conv2"),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, 28, 28, 1))

model.summary()

# === Step 6: Train CNN Model (A2) ===
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# === Step 7: Plot Training & Validation Loss (A3) ===
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Training vs Validation Loss")
plt.show()

# === Step 8: Evaluate Model Accuracy (A4) ===
score = model.evaluate(X_test, y_test, verbose=1)
print('\nCNN Model Accuracy:', score[1])

# === Step 9: Feature Map Visualization (A1) ===
sample_image = X_test[0].reshape(1, 28, 28, 1)

model.predict(sample_image)

layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name]
feature_map_model = Model(inputs=model.input, outputs=layer_outputs)

feature_maps = feature_map_model.predict(sample_image)

for layer_idx, feature_map in enumerate(feature_maps):
    num_filters = feature_map.shape[-1]
    
    plt.figure(figsize=(15, 5))
    plt.suptitle(f"Feature Maps - Layer {layer_idx+1}", fontsize=14)

    for i in range(min(num_filters, 6)):
        plt.subplot(1, 6, i+1)
        plt.imshow(feature_map[0, :, :, i], cmap='gray')
        plt.axis('off')
        plt.title(f"Filter {i+1}")

    plt.show()

# === Step 10: Design & Train Fully Connected Model (A7) ===
fc_model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

fc_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

fc_history = fc_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# === Step 11: Plot Training vs Validation Loss (A8) ===
plt.plot(fc_history.history['loss'], label='Training Loss')
plt.plot(fc_history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Fully Connected Network - Training vs Validation Loss")
plt.show()

# === Step 12: Test & Evaluate Fully Connected Model (A8) ===
fc_score = fc_model.evaluate(X_test, y_test, verbose=1)
print('\nFully Connected Network Accuracy:', fc_score[1])
